In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [3]:
import numpy as np
np.random.seed(13)

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

import gensim

Using TensorFlow backend.


In [4]:
path = '../input/alice.txt'
corpus = open(path).readlines()[:300]
corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1
dim = 100
window_size = 2

In [5]:
def generate_data(corpus, window_size, V):
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            contexts = []
            labels   = []            
            s = index - window_size
            e = index + window_size + 1
            
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)

            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            y = np_utils.to_categorical(labels, V)
            yield (x, y)


In [6]:
cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(V, activation='softmax'))


Instructions for updating:
Colocations handled automatically by placer.


In [7]:
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')


In [8]:
for ite in range(10):
    loss = 0.
    for x, y in generate_data(corpus, window_size, V):
        loss += cbow.train_on_batch(x, y)

    print(ite, loss)

Instructions for updating:
Use tf.cast instead.
0 17409.690202236176
1 16148.344410419464
2 16017.57739496231
3 15922.420579314232
4 15814.984168946743
5 15715.434266448021
6 15628.712351620197
7 15551.12812909484
8 15476.529289752245
9 15402.650278761983


In [10]:
d={}
# f = open('vectors.txt' ,'w')
# f.write('{} {}\n'.format(V-1, dim_embedddings))
vectors = cbow.get_weights()[0]
for word, i in tokenizer.word_index.items():
    d[word]=list(vectors[i, :])


In [14]:
print(d.keys())

dict_keys(['the', 'she', 'to', 'and', 'it', 'was', 'a', 'of', '’', 'in', 'alice', 'her', 'i', 'that', 'down', 'you', 'for', 'but', 'very', 'on', 'had', 'this', 'as', 'be', 'so', 'little', 'with', 'way', 'at', 'out', 'not', 'or', 'herself', 'up', 'how', 'like', 'one', 'no', 'rabbit', 'when', 'all', 'see', 'into', 'what', 'there', 'think', 'me', 'if', 'thought', 'could', 'time', 'about', 'went', 'were', 'off', 'said', 'by', 'get', 'do', 'would', 'much', 'feet', 'through', 'door', 'alice’s', 'is', 'use', 'nothing', 'say', 'found', 'wonder', 'they', 'which', 'key', '‘and', 'shall', 'then', 'either', 'moment', 'going', 'too', '‘i', 'my', 'eat', 'ever', 'came', 'hall', 'garden', 'go', 'poor', 'once', 'well', 'suddenly', 'white', 'quite', 'looked', 'never', 'before', 'after', 'large', 'again', 'look', 'tried', 'great', 'must', 'things', 'good', 'right', 'began', 'them', 'might', 'table', 'any', 'now', 'adventures', 'wonderland', 'ebook', 'under', 'hole', 'book', 'getting', '‘oh', 'dear', 'oh'

In [16]:
print(d['voice'])

[0.022111446, -0.024869874, -0.05780252, -0.016599277, 0.012027467, -0.029446954, 0.027906682, 0.026086615, 0.0709466, 0.03227715, -0.07704123, -0.04758616, -0.0034261788, -0.026065022, 0.020674929, -0.014053873, -0.0070777936, -0.076414004, -0.016559005, -0.093848206, -0.0015646361, 0.09082728, -0.01663035, -0.03771302, -0.03024775, 0.010789503, -0.046133325, 0.029330084, 0.084216386, 0.025789026, -0.078797825, -0.06275805, -0.056608908, 0.039977673, 0.0178531, 0.023548504, -0.005291233, -0.05733729, 0.07807205, -0.065886825, 0.05576106, -0.04774684, 0.0030095957, 0.058414146, 0.06489876, 0.062968135, 0.07425928, -0.0098067345, 0.0001252573, -0.07118572, 0.09541443, 0.095160246, 0.027523402, -0.016431738, 0.0073803943, -0.0053849304, -0.05143182, -0.069541804, -0.011154982, 0.08308398, 0.03513116, 0.09122274, 0.020411184, -0.09466915, 0.008611854, 0.018507257, 0.030426554, 0.06386126, 0.097904675, 0.0058025965, -0.09679569, 0.06642695, -0.008525913, 0.071977615, 0.0915481, -0.05396860

In [17]:
def avg_sentence_vector(words, model, num_features):
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0
    for word in words:
        if word in model.keys():
            featureVec = np.add(featureVec, model[word])

    return featureVec

In [22]:
sentence1 = ['i','will','be','eating','tea']
sentence2 = ['i','will','be','drinking','coffee']

In [23]:
feature1 = avg_sentence_vector(sentence1,d,100)
feature2 = avg_sentence_vector(sentence2,d,100)

In [24]:
from scipy.spatial.distance import cosine
print(cosine(feature1,feature2))

0.003548562526702881


In [28]:
sentence3 = ['I','am','walking','to','America']
sentence4 = ['I','am','going','to','Bharat']

In [29]:
feature3 = avg_sentence_vector(sentence3,d,100)
feature4 = avg_sentence_vector(sentence4,d,100)

In [30]:
print(cosine(feature3,feature4))

0.02755063772201538
